In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import flopy
import config

In [ ]:
lx = 100.
ly = 100.
nlay = 1
nrow = 20
ncol = 20
delr = lx / ncol
delc = ly / nrow
top = 0
botm = -100
nper = 1
nstp = 1
ibound = np.ones((nlay, nrow, ncol), dtype=np.int)
ibound[0, 0, 0] = -1
ibound[0, nrow - 1, ncol - 1] = -1
strt = np.zeros((nlay, nrow, ncol), dtype=np.float)
strt[0, 0, 0] = 1.

In [ ]:
model_ws = './work/exFlopyIntroA'
name = 'fpintro'
m = flopy.modflow.Modflow(modelname=name, model_ws=model_ws, 
                         exe_name=config.mfexe)
dis = flopy.modflow.ModflowDis(m, nlay, nrow, ncol, nper,
                              delr, delc, top=top, botm=botm)
bas = flopy.modflow.ModflowBas(m, ibound=ibound, strt=strt)
lpf = flopy.modflow.ModflowLpf(m, ipakcb=52)
pcg = flopy.modflow.ModflowPcg(m)
oc = flopy.modflow.ModflowOc(m, stress_period_data={(0, 0): 
                                                    ['save head',
                                                    'save budget']}, )
m.write_input()
m.run_model()

In [ ]:
fname = model_ws + '/' + name + '.hds'
hdobj = flopy.utils.HeadFile(fname)
head = hdobj.get_data()

fname = model_ws + '/' + name + '.cbc'
budobj = flopy.utils.CellBudgetFile(fname)
frf = budobj.get_data(text='FLOW RIGHT FACE')[0]
flf = budobj.get_data(text='FLOW FRONT FACE')[0]

In [ ]:
mm = flopy.plot.ModelMap(model=m)
mm.plot_grid()
mm.plot_ibound()
mm.plot_array(head, cmap='jet')
mm.contour_array(head)
mm.plot_discharge(frf, frf)